In [1]:
'''In this game, there is a list of movie names present, out of which our interpreter will choose 1 random movie.
The user first has to input their guesses on the besis of given clues and then, will be asked to guess any alphabet. 
If the random movie contains that alphabet, it will be shown as the output(with correct placement) else the program will ask
you to guess another alphabet. The user will be given 6 turns and total 3 clues on wrong guess to guess
the complete word.'''
#requirements-
#1. the movie list and clues like movie dialogue/actor name/no. of words in movie name
#2. create list/dict for random selection and info related to chosen movie
#3. regex to differentciate clues from webscrapped data
#4. convert this data and save it in pickle formate
#4. word analysis to match capital and small alphabate inputs to output on chosen movie to avoid errors 

'In this game, there is a list of movie names present, out of which our interpreter will choose 1 random movie.\nThe user first has to input their guesses on the besis of given clues and then, will be asked to guess any alphabet. \nIf the random movie contains that alphabet, it will be shown as the output(with correct placement) else the program will ask\nyou to guess another alphabet. The user will be given 6 turns and total 3 clues on wrong guess to guess\nthe complete word.'

In [1]:
import requests
from bs4 import BeautifulSoup

#website which provide movie dialogues ,names and actor information for clues
url="https://entertainment.expertscolumn.com/100-greatest-dialogues-bollywood-movies"

response = requests.get(url)
#print(response)

response=response.content

soup=BeautifulSoup(response,'html.parser')

text_data=soup.findAll('ol')

for points in soup.find_all('ol'):
    point = str(points.text)
    #print(point)

#datatype of pint is str

import re
#regex pattern for dialogues
pattern= '\\n(\S*[^\(]*)'
dialg=re.findall(pattern,point)
#dialg
#regex pattern for three element which is movie name, actor name and year from data
pattern_1='\(\w*[^\)]*\)'
three_elm=re.findall(pattern_1,point)
#three_elm

#to create dictonary we need to match dialg and three_elm list for key-value pair
#in dialg last item is empty
#len(dialg)

#removed last item 
dialg= dialg[:-1]

#dict from dialg and three_elm for key-value pair
res = dict(zip(dialg, three_elm))
#print(res)

#following movies need to delete because of extra or less dashes in data -
#(this will helpful for future regex funtion to seperate movie name and actor names)

#"Gehne tudvao, gehne banvao, aur koriyaan khelo... so aaram se” (Sahib, Biwi aur Ghulam- Rehman- 1962)'
#"Phool toh murjha jaate hain, lekin kaante daman thham lete hain," (Mughal-e-Azam- Madhubala- 1960)'
#“Naam Vijay Deenanath Chauhan, Baap Ka naam Deenanath Chauhan” (Agneepath- 1990)'
#'Insan naam mein mazab dhoond leta hai ': '(A Wednesday! Naseeruddin Shah- 2008)'

del res['“Naam Vijay Deenanath Chauhan, Baap Ka naam Deenanath Chauhan” ']  
del res['"Phool toh murjha jaate hain, lekin kaante daman thham lete hain," ']
del res['"Gehne tudvao, gehne banvao, aur koriyaan khelo... so aaram se” ']
del res['Insan naam mein mazab dhoond leta hai ']
del res['“Mahulle ke laundon ka pyar ... aksar doctor aur engineer uthake le jaate hai” ']
del res['“Insaan apne liye gaye faislo se hi bada hota hai” ']


In [2]:
import pickle

serialized = pickle.dumps(res) 

with open('file.pkl', 'wb') as file: 
# A new file will be created 
    pickle.dump(res, file)



In [ ]:

#this dict is ready to use for random selection

def main():
        import random
        import itertools
        import pickle
        with open('file.pkl', 'rb') as file:
            res_pkl = pickle.load(file)
    
        #converting random choice in list formate for indexing to get movie dialogue
        clue_dialg=random.choice(list(res_pkl.items()))
        #print("list:",clue_dialg)
        #print("movie_dialogue:",clue_dialg[0])
        
        #again need to convert list(clue_dialg) to str to use regex 
        movie_p=' \'\((\w*[^\-]*)'
        
        choice_movie=re.findall(movie_p,str(clue_dialg))
        # for removing any space for display error
        s_choice_movie1=''
        for x in choice_movie:
            s_choice_movie1+=' '+ x
        s_choice_movie=s_choice_movie1.replace(" ", "")
        #print("movie_name:",s_choice_movie)
        
        #for movie words count clue
        import shlex
        words = shlex.split(s_choice_movie1)
        word_count = len(words)
        #print("movie_words:",word_count)
        
        #for actor and year clue
        clue_p='\-(\w*\W*[^\)]*)'
        clue_actor_year=re.findall(clue_p,str(clue_dialg))
        #print("actor&year:",clue_actor_year)
        
        #for year clue
        s = ''.join(str(i) for i in clue_actor_year)
        s_p='\d{4}'
        clue_year=re.findall(s_p,s)
        #print("year:",clue_year)
        
        stages=['''
        =============
         you loose!!! 
        =============
        ''','''
        ===
        |*|
        ===
        ''','''
        =====
        |*|*|
        =====
        ''','''
        =======
        |*|*|*|
        =======
        ''','''
        =========
        |*|*|*|*|
        =========
        ''','''
        ===========
        |*|*|*|*|*|
        ===========
        ''','''
        =============
        |*|*|*|*|*|*|
        =============
        ''']
        
        lives=6
        display_ =[]
        for i in range(len(s_choice_movie)):
            display_ += '_'
        print(display_)
        
        #movie dialogue clue
        print('Movie Dialogue :',clue_dialg[0])
        
        game_over=False
        
        #intertools to match input in display whether its capital or small alphabate
        while not game_over:
            guessed_letter=input("Guess a letter:")
            x=list(map(''.join, itertools.product(*zip(guessed_letter.upper(), guessed_letter.lower()))))
            
            for position in range(len(s_choice_movie)):
                letter=s_choice_movie[position]
                if letter in x:
                    display_[position]=letter
            print(display_)
            if (x[0] not in s_choice_movie) and (x[1] not in s_choice_movie):
                lives-=1
                if lives==5:
                    print("words in movie name:",word_count)
                if lives==4:
                    print("year of relesed:",clue_year)
                if lives==3:
                    print("movie actor & year of relesed:",clue_actor_year)
                
            if lives==0:
                game_over=True
                print("You Lose!!!")
                print(s_choice_movie)
                    
            if '_' not in display_:
                game_over=True
                print("You Win!!!")
                
            print(stages[lives])
                
        repeat=input("play again?").lower()
        if repeat=='yes':
            main()
        else:
            print("exit")
            exit()
main()
         

['_', '_', '_', '_', '_', '_', '_']
Movie Dialogue : “Dil to har kisi ke paas hota hai, lekin sab, Dilwale nahin hote” 


Guess a letter: d


['D', '_', '_', '_', '_', '_', '_']

        |*|*|*|*|*|*|
        


Guess a letter: o


['D', '_', '_', '_', '_', '_', '_']
words in movie name: 1

        |*|*|*|*|*|
        


...

['K', 'a', 'l', 'H', '_', '_', 'a', 'H', '_']

        =====
        |*|*|
        =====
        


Guess a letter: o


['K', 'a', 'l', 'H', 'o', '_', 'a', 'H', 'o']

        =====
        |*|*|
        =====
        


Guess a letter: n


['K', 'a', 'l', 'H', 'o', 'N', 'a', 'H', 'o']
You Win!!!

        =====
        |*|*|
        =====
        
